In [1]:
import nltk
import random
import pandas as pd
import matplotlib.pyplot as plt
import re

In [17]:
train = pd.read_csv("text_emotion.csv", usecols = ['content','sentiment'])

In [18]:
train.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
ohe = OneHotEncoder()
random = pd.get_dummies(train['sentiment'])
random.head()

,anger,boredom,empty,enthusiasm,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0


In [22]:
train = pd.concat([train, random], axis = 1)
train.head()

,sentiment,content,anger,boredom,empty,enthusiasm,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
0,empty,@tiffanylue i know i was listenin to bad habi...,0,0,1,0,0,0,0,0,0,0,0,0,0
1,sadness,Layin n bed with a headache ughhhh...waitin o...,0,0,0,0,0,0,0,0,0,0,1,0,0
2,sadness,Funeral ceremony...gloomy friday...,0,0,0,0,0,0,0,0,0,0,1,0,0
3,enthusiasm,wants to hang out with friends SOON!,0,0,0,1,0,0,0,0,0,0,0,0,0
4,neutral,@dannycastillo We want to trade with someone w...,0,0,0,0,0,0,0,0,1,0,0,0,0


In [23]:
train.drop(columns = ['sentiment'], inplace=True)

In [27]:
print(train.shape[0])

40000


In [28]:
targets = ["anger","boredom","empty","enthusiasm","fun","happiness","hate","love","neutral","relief","sadness","surprise","worry"]

In [29]:
def cleaning(sen):
    sen = re.sub(r"what's","what is",sen)
    sen = re.sub(r"\'s'"," ",sen)
    sen = re.sub(r"\'ve'"," have ",sen)
    sen = re.sub(r"can't","cannot",sen)
    sen = re.sub(r"n't"," not ",sen)
    sen = re.sub(r"i'm","i am ",sen)
    sen = re.sub(r"\'re'"," are ",sen)
    sen = re.sub(r"\'d"," would ",sen)
    sen = re.sub(r"\'ll","will",sen)
    sen = re.sub(r"\'scuse", "excuse",sen)
    sen = re.sub("\W"," ",sen)
    sen = re.sub(r"\s+"," ",sen)
    sen = sen.strip(' ')
    return sen

In [30]:
cleaned_training_data = []
for i in range(len(train)):
    cleaned_comment = cleaning(train["content"][i])
    cleaned_training_data.append(cleaned_comment)
train["content"] = pd.Series(cleaned_training_data).astype(str)

In [31]:
train.head()

,content,anger,boredom,empty,enthusiasm,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
0,tiffanylue i know i was listenin to bad habit ...,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Layin n bed with a headache ughhhh waitin on y...,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Funeral ceremony gloomy friday,0,0,0,0,0,0,0,0,0,0,1,0,0
3,wants to hang out with friends SOON,0,0,0,1,0,0,0,0,0,0,0,0,0
4,dannycastillo We want to trade with someone wh...,0,0,0,0,0,0,0,0,1,0,0,0,0


In [33]:
from sklearn.model_selection import train_test_split

In [53]:
length = train.shape[0]

In [55]:
int(length*0.8)

32000

In [59]:
X_train.shape

(32000,)

In [60]:
X_train = train.content[0:int(length*0.8)]
X_test = train.content[int(length*0.8):int(length + 1)]

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(max_features=15, min_df=2)

In [62]:
X_train_dtm = vector.fit_transform(X_train)
X_test_dtm = vector.fit_transform(X_test)

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model_log = LogisticRegression(C=6.0)

In [82]:
submission = train[int(length*0.8):length]

In [83]:
submission.shape

(8000, 14)

In [86]:
for label in targets:
    print("...On the label:{}".format(label))
    y = train[label][0:int(length*0.8)]
    # train the model using X_train_dtm and y
    model_log.fit(X_train_dtm, y)
    # computing the training accuracy
    predictions = model_log.predict(X_train_dtm)
    print("Training accuracy: {}".format(accuracy_score(y, predictions)))
    # compute the predicted probabilities for X_test_dtm
    probability = model_log.predict_proba(X_test_dtm)[:,1]
    submission[label] = probability

...On the label:anger
Training accuracy: 0.9969375
...On the label:boredom
Training accuracy: 0.99496875
...On the label:empty
Training accuracy: 0.97815625


D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

...On the label:enthusiasm
Training accuracy: 0.98228125
...On the label:fun
Training accuracy: 0.96103125
...On the label:happiness
Training accuracy: 0.8926875
...On the label:hate


D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

Training accuracy: 0.962
...On the label:love
Training accuracy: 0.925375
...On the label:neutral
Training accuracy: 0.7878125
...On the label:relief
Training accuracy: 0.9646875
...On the label:sadness


D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

Training accuracy: 0.8473125
...On the label:surprise
Training accuracy: 0.94553125
...On the label:worry
Training accuracy: 0.76121875


D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda_installation\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [87]:
submission.head()

,content,anger,boredom,empty,enthusiasm,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
32000,ReBeLR My day was cool a lil twisted I wo not ...,0.004153,0.006029,0.017673,0.009308,0.025020,0.056135,0.039942,0.044529,0.148030,0.039449,0.189499,0.050032,0.357893
32001,is on air,0.002797,0.004847,0.022201,0.020459,0.044390,0.138154,0.034435,0.065648,0.231022,0.036110,0.123153,0.064849,0.213445
32002,greenpolkadot99 u go to profile then ill help ...,0.001824,0.006310,0.026510,0.031125,0.039602,0.105747,0.031666,0.056261,0.251819,0.033291,0.151225,0.041304,0.223382
32003,Platypuses are the only mammals that can lay eggs,0.002332,0.002908,0.015706,0.011815,0.042186,0.110098,0.053409,0.091216,0.173826,0.034453,0.151798,0.081620,0.223703
32004,loadedman look at my background that s how I w...,0.003657,0.002683,0.014409,0.010397,0.031897,0.081884,0.048450,0.086958,0.125450,0.030676,0.205284,0.060375,0.309756


In [88]:
submission.to_csv("submission_covid.csv", index=False)

In [21]:
my_submission = pd.read_csv("submission_logreg_2.csv")

In [24]:
my_submission.shape

(11, 7)